In [138]:
from sortedcontainers import SortedList
from math import sqrt
from scipy.stats import mannwhitneyu,norm

In [196]:

class seq_mann_whitney_U :
    def __init__(self) :
        self.X = SortedList()
        self.Y = SortedList()
        self.U = None
        self.t = None
    def add_x(self,val) :
        if len(self.Y) != 0 :       
            if self.U == None :
                self.U = 0.0
            lb = self.Y.bisect_left(val)
            rb = self.Y.bisect_right(val)
            if lb != rb :
                self.U += 0.5
            self.U += lb
        self.X.add(val)
    def add_y(self,val) :
        if len(self.X) != 0 :       
            if self.U == None :
                self.U = 0.0
            lb = self.X.bisect_left(val)
            rb = self.X.bisect_right(val)
            if lb != rb :
                self.U += 0.5
            self.U += len(S.X) - rb
        self.Y.add(val)
    def asymptotic_p(self) :
        if self.U == None :
            return None
        nx = len(self.X)
        ny = len(self.Y)
        mu = nx*ny/2
        sigma = sqrt(mu*(nx+ny+1)/6)
        return norm.pdf((self.U - mu)/sigma)
        
    
    
        

In [204]:
S = seq_mann_whitney_U()

In [205]:
Y = [19,22,16,29,24]
X = [20,11,17,12]
for y in Y :
    S.add_y(y)
for x in X :
    S.add_x(x)

In [206]:
print(S.X)
print(S.Y)
print(S.U)
print(S.asymptotic_p())

SortedList([11, 12, 17, 20])
SortedList([16, 19, 22, 24, 29])
3.0
0.09172699738279745


In [69]:
from scipy.stats import mannwhitneyu

In [180]:
mannwhitneyu(Y,X,method="asymptotic")

MannwhitneyuResult(statistic=3.0, pvalue=0.11134688653314041)

In [171]:
norm.pdf(-1.715,0,1)

0.0916708317993663

In [191]:
norm.pdf(-1.7146428199482247,0,1)

0.09172699738279745

In [208]:
norm.pdf(0.3,0,1)

0.3969525474770118